# Предобученные сети, которыми можно пользоваться

## LeNet'98

LeNet была создана ,чтобы распознавать цифры на почтовых бланках.

Особенности LeNet5:

- Свёрточная нейросеть, использующая последовательность из трёх слоёв: слои свёртки (convolution), слои группирования (pooling) и слои нелинейности (non-linearity) –> с момента публикации работы Лекуна это, пожалуй, одна из главных особенностей глубокого обучения применительно к изображениям.
- Подвыборка с использованием усреднения карт.
- Нелинейность в виде гиперболического тангенса или сигмоид (проблема с затухающим градиентом).
- Финальный классификатор в виде многослойной нейросети.

<img src='https://drive.google.com/uc?exoprt=view&id=1pPe1aBh7ySg89cxbWEZ07iabvvXABUxd'>

Многие предобученные модели обучались на датасете ImageNet, который содержит 14,197,122 картинок - это набор данных размечанных изображений с высоким разрешением, относящихся примерно к 22 тысячам категорий. Проводился «Крупномасштабный конкурс визуального распознавания ImageNet» (ILSVRC2013). ILSVRC использует подмножество ImageNet из примерно 1000 изображений в каждой из 1000 категорий.

<img src='https://avatars.mds.yandex.net/get-zen_doc/127510/pub_5c33ad37c906e200abbace3b_5c33adfbe5e73b00aad095a1/scale_1200'>

## AlexNet'12

В AlexNet результаты вычислений LeNet масштабированы в гораздо более крупную нейросеть, которая способна изучить намного более сложные объекты и их иерархии. Особенности:

- Использование блоков ReLU в качестве нелинейностей (нет проблемы с затуханием градиентов).
- Использование max pooling, что позволяет избежать эффектов усреднения average pooling.

На вход идут картинки 224х224, естественно не все картинки такого размера, поэтому будет достаточно просто сжать их до нужного размера.

<img src='https://drive.google.com/uc?export=view&id=1sjEftFGiJ50-m3VevamktVznsx6bY3Yw' width=700>

## VGG'14
В разработанных в Оксфорде VGG-сетях в каждом свёрточном слое впервые применили фильтры 3х3 и объединили эти слои в последовательности свёрток.

Вместо применяемых в AlexNet фильтров 9х9 и 11х11 стали применять гораздо более мелкие фильтры, которых старались избежать авторы LeNet. Но большим преимуществом VGG стала находка, что несколько свёрток 3х3, объединённых в последовательность, могут эмулировать более крупные свертки, например, 5х5 или 7х7 (7х7 + 1 байес = 50 обучаемых параметров).

Каскад из двух сверток 3х3 равен свертке 5х5, но с меньшим количеством параметров.
(5х5 = 25 + 1 = 26; 3x3 + 3x3 + 2 = 20)

<img src='https://drive.google.com/uc?export=view&id=1GvrtEDocJ3xp9RKqgQu0-JnyTssqZhzV'>

Глубокие сверточные нейронные сети превзошли человеческий уровень классификации изображений в 2015 году. Глубокие сети извлекают низко-, средне- и высокоуровневые признаки  сквозным многослойным способом, а увеличение количества слоев обогатить «уровни» признаков. Но у глубоких нейронных сетей была проблема: затухающие градиенты. Особенно это явно чувствуется с сигмоидой.

$d\sigma = \sigma(1 - \sigma) \leqslant \frac{1}{4}$

<img src='https://drive.google.com/uc?export=view&id=171JbyNkSSqzhPdX4fp439zOouEzJmq_s'>

Если постоянно брать производную сигмоиды, то максимальная производная сигмоиды - это 0,25. Если мы будем идти к началу чети, пто 0,25 будет возводиться в степень сколько у нас сигмоид. И это быстро становится близким к нулю. А если градиент равен нулю, то никакого обучения не будет.

Кроме того, что можно вместо сигмоиды брать ReLu, можно пробрасывать ошибку.

## GoogLeNet

Эта сеть использует Inception блоки. Это параллельная комбинация свёрточных фильтров 1х1, 3х3 и 5х5. Но главная особенность заключается в использовании свёрточных блоков 1х1 для уменьшения количества каналов перед подачей в более «дорогие» сверточные блоки. Обычно эту часть называют bottleneck. Вместо использования свертки 5х5 на нашем изображении, можем сначала пройтись сверткой 1х1 уменьшив количество каналов, а затем по ним пройтись сверткой 5х5, вернув количество каналов. Операций будет меньше, а результат будет одинаковый.

<img src='https://drive.google.com/uc?export=view&id=1hgoTi6d-pdRPHgnfVGssQIQXBUdrkWrk'>

In [ ]:
!pip install torchsummary 

In [ ]:
import torch
from torch import nn
from torchsummary import summary

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
model = nn.Sequential(
    nn.Conv2d(in_channels=256,
              out_channels=256,
              kernel_size=5)
)

summary(model.to(device), input_size=(256, 16, 16))

In [ ]:
model = nn.Sequential(
    nn.Conv2d(in_channels=256,
              out_channels=128,
              kernel_size=1),
    nn.Conv2d(in_channels=128,
              out_channels=256,
              kernel_size=5)
)

summary(model.to(device), input_size=(256, 16, 16))

Разработчики этой сети придумали способ, чтобы градиенты не затухали: вводится несколько вспомогательных функций на протяжении всей сети, чтобы когда градиент от первого выхода начинал затухать, подключался градиент со второго выхода.

<img src='https://drive.google.com/uc?export=view&id=1q3oJXpwGStYit5Ii13DIsexVqxwIjyjE'>

## ResNet'15

ResNet лучше всех борется с проблемами затухающих градиентов.

До этого боролись с затухающими градиентами только за счет ввода другой функции активации.

Чтобы преодолеть проблему затухающих градиентов, Microsoft ввела глубокую «остаточную» структуру обучения.

<img src='https://drive.google.com/uc?export=view&id=1RGJQl4-SmysYbAqwcy8Lm5qEPbiebZOO'>

Смысл:

$y = f(x) + x $<br>
$dy = df(x) + 1 $<br>
<h3>$\frac{dL}{dx} = \frac{dL}{dy} \frac{dy}{dx} = \frac{dL}{dy}(df(x) + 1 )$</h3><br>

То есть градиенты всё равно будут протекать дальше в немного измененном виде.


Соединения быстрого доступа (shortcut connections, residual connections) пропускают один или несколько слоев и выполняют сопоставление идентификаторов.

<img src='https://drive.google.com/uc?export=view&id=1JcQDIjA-97L2xs3o-JD4SWBld9J0OMW-'>

Еще одна особенность ResNet, что в конце отсутствуют полносвязные слои, причина в том, что имеется уже и так довольно сложная нейронная сеть, в которой уже и так могла решиться задача, которая обычна перекладывается на полносвязный слой.

<img src='https://drive.google.com/uc?export=view&id=1qqbZ6iWZaD6LMjuIJ85mBGYpBwi0w-HL'>

## Распознование алфавита языка жестов

In [ ]:
from torchvision import models, transforms, datasets
import torch
from torch import nn
import matplotlib.pyplot as plt

In [ ]:
train_data_path = '../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/'

Определим трансформации для тестового и обучающего датасета.



In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize(224),
#     transforms.RandomAffine(degrees=5, scale=(0.3, 1.1)), # аугументации
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

Загрузим датасет и поделим его на обучение и валидацию.



In [ ]:
%%time
# данные разложены по папкам, берём из папок данные
train_dataset = datasets.ImageFolder(train_data_path, transform=train_transforms)

In [ ]:
%%time
val_dataset = datasets.ImageFolder(train_data_path, transform=test_transforms)

In [ ]:
torch.manual_seed(1)
num_train_samples = len(train_dataset)
# num_train_samples = 20000

val_split = 0.2
split = int(num_train_samples * val_split)

# перемешиваем индексы
indices = torch.randperm(num_train_samples)

train_subset = torch.utils.data.Subset(train_dataset, indices[split:])
val_subset = torch.utils.data.Subset(val_dataset, indices[:split])

len(train_subset), len(val_subset)

In [ ]:
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    dataset=train_subset, 
    batch_size=batch_size,
    shuffle=True
)

val_dataloader = torch.utils.data.DataLoader(
    dataset=val_subset,
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
classes = train_dataloader.dataset.dataset.classes

Можем проверить, как работает dataloader.

In [ ]:
for img, label in train_dataloader:
    print(img.shape, label.shape)
    print(f'Ground Truth {classes[label[0]]}')
    plt.imshow(img[0].permute(1, 2, 0))
    break

In [ ]:
resnet = models.resnet50(pretrained=True)
summary(resnet.to(device), input_size=(3, 224, 224))

Заморозим претренерованные слои, чтобы он не обучались.



In [ ]:
for param in resnet.parameters():
    param.requires_grad = False

Посмотрим кол-во параметров по слоям.

In [ ]:
resnet

Нам нужно переопределить последний слой т.к. у нас не 1000 классов как в ResNet.

Для этого берём количество парамтеров, которое идёт на вход последнего линейного слоя и на выход даём количество классов и меняем в реснете на новый полносвязный слой.

In [ ]:
in_features = resnet.fc.in_features
fc = nn.Linear(in_features=in_features, out_features=len(classes))
resnet.fc = fc

summary(resnet.to(device), input_size=(3, 224, 224))

Теперь у нас на последнем слое не 1000, а 29 выходов и обучаем мы только последний слой.

Зададим функцию потерь и оптимизатор.

In [ ]:
# функция потерь
criterion = nn.CrossEntropyLoss()

# оптимизатор
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)

Обучаем модель.

In [ ]:
from time import time
from tqdm import tqdm


def train(model,
          criterion,
          optimizer,
          train_dataloader,
          test_dataloader,
          print_every,
          num_epoch):
    steps = 0
    train_losses, val_losses = [], []

    model.to(device)
    for epoch in tqdm(range(num_epoch)):
        running_loss = 0
        correct_train = 0
        total_train = 0
        start_time = time()
        iter_time = time()
        
        model.train()
        # проходимся по обучающему датасету
        for i, (images, labels) in enumerate(train_dataloader):
            steps += 1
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            # получаем предсказания
            output = model(images)
            # считаем ошибку
            loss = criterion(output, labels)

            # насколько корректно всё предсказали
            correct_train += (torch.max(output, dim=1)[1] == labels).sum()
            # сколько всего объектов было в предсказании
            total_train += labels.size(0)

            # Backward and optimize - обучение
            # обнуляем градиенты
            optimizer.zero_grad()
            # считаем градиенты
            loss.backward()
            # меняем веса
            optimizer.step()

            running_loss += loss.item()

            # Logging
            if steps % print_every == 0:
                print(f'Epoch [{epoch + 1}]/[{num_epoch}]. Batch [{i + 1}]/[{len(train_dataloader)}].', end=' ')
                print(f'Train loss {running_loss / steps:.3f}.', end=' ')
                print(f'Train acc {correct_train / total_train * 100:.3f}.', end=' ')
                
                # считаем, что происходит на валидации (градиент не трогаем - так быстрее)
                with torch.no_grad():
                    # переводим модель в режим валидации
                    model.eval()
                    correct_val, total_val = 0, 0
                    val_loss = 0
                    # проходимся по тестовому набору данных
                    for images, labels in test_dataloader:
                        images = images.to(device)
                        labels = labels.to(device)
                        # предсказания
                        output = model(images)
                        loss = criterion(output, labels)
                        val_loss += loss.item()

                        correct_val += (torch.max(output, dim=1)[1] == labels).sum()
                        total_val += labels.size(0)

                print(f'Val loss {val_loss / len(test_dataloader):.3f}. Val acc {correct_val / total_val * 100:.3f}.', end=' ')
                print(f'Took {time() - iter_time:.3f} seconds')
                iter_time = time()

                train_losses.append(running_loss / total_train)
                val_losses.append(val_loss / total_val)


        print(f'Epoch took {time() - start_time}') 
        torch.save(model, f'checkpoint_{correct_val / total_val * 100:.2f}')
        
    return model, train_losses, val_losses

In [ ]:
# печатаем каждые 50 итераций
print_every = 50
# обучаем 2 эпохи (2 ч 15 мин)
num_epoch = 2

resnet, train_losses, val_losses = train(
    model=resnet,
    criterion=criterion,
    optimizer=optimizer,
    train_dataloader=train_dataloader,
    test_dataloader=val_dataloader,
    print_every=print_every,
    num_epoch=num_epoch
)

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()

Инференс

Посмотрим как работает на тестовых картинках, на которых модель не обучалась.

In [ ]:
from pathlib import Path
from PIL import Image


test_data_path = Path('../input/asl-alphabet/asl_alphabet_test/asl_alphabet_test/')


class ASLTestDataset(torch.utils.data.Dataset):
    def __init__(self, root_path, transforms=None):
        super().__init__()
        
        # тестовы трансформации
        self.transforms = transforms
        # считываем все пути до картинок
        self.imgs = sorted(list(Path(root_path).glob('*.jpg')))
        
    def __len__(self):
        # сколько всего есть картинок
        return len(self.imgs)
    
    def __getitem__(self, idx):
        # получаем объекты по индексу
        img_path = self.imgs[idx]
        # открываем картинку
        img = Image.open(img_path).convert('RGB')
        # в названии картинки хранится класс, извекаем его с помощью сплита
        label = img_path.parts[-1].split('_')[0]
        if self.transforms:
            # если есть трансформация - применяем её
            img = self.transforms(img)
        
        # возвращаем картинку и класс
        return img, label

In [ ]:
# инициализируем тестовый датасет с тестовыми трансформациями
test_dataset = ASLTestDataset(test_data_path, transforms=test_transforms)

columns = 7
row = round(len(test_dataset) / columns)

fig, ax = plt.subplots(row, columns, figsize=(columns * row, row * columns))
plt.subplots_adjust(wspace=0.1, hspace=0.2)

i, j = 0, 0
# проходимся по тестовому датасету и визуализируем,
# что предсказала модель и что должна была предсказать
for img, label in test_dataset:
    img = torch.Tensor(img)
    img = img.to(device)
    resnet.eval()
    prediction = resnet(img[None])

    ax[i][j].imshow(img.cpu().permute(1, 2, 0))
    ax[i][j].set_title(f'GT {label}. Pred {classes[torch.max(prediction, dim=1)[1]]}')
    ax[i][j].axis('off')
    j += 1
    if j == columns:
        j = 0
        i += 1
        
plt.show()